In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [2]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3


In [3]:
filenames=os.listdir("./dataset/training_set")
categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)
df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0

In [5]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
#To prevent over fitting we will stop the learning after 10 epochs and val_loss value not decreased
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001) 
# reduce the learning rate when then accuracy not increase for 2 steps
callbacks = [earlystop,learning_rate_reduction]

In [6]:
df["category"] = df["category"].replace({0:'cat',1:'dog'}) # convert 1 to dog and 0 to cat
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15
df

,filename,category
0,cat.1.jpg,cat
1,cat.10.jpg,cat
2,cat.100.jpg,cat
3,cat.1000.jpg,cat
4,cat.1001.jpg,cat
...,...,...
7997,dog.996.jpg,dog
7998,dog.997.jpg,dog
7999,dog.998.jpg,dog
8000,dog.999.jpg,dog


In [7]:
#Training and validation data generator:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "./dataset/training_set", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "./dataset/training_set", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

Found 6399 validated image filenames belonging to 2 classes.
Found 1601 validated image filenames belonging to 2 classes.


C:\ProgramData\Anaconda3\envs\gputest\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:273: UserWarning: Found 2 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [8]:
# Model training
epochs=3
history = model.fit(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 426 steps, validate for 106 steps
Epoch 1/3
426/426 [==============================] - 200s 469ms/step - loss: 0.9849 - accuracy: 0.5749 - val_loss: 0.9822 - val_accuracy: 0.5780
Epoch 2/3
426/426 [==============================] - 70s 165ms/step - loss: 0.6651 - accuracy: 0.6482 - val_loss: 0.5832 - val_accuracy: 0.7006
Epoch 3/3
426/426 [==============================] - 62s 145ms/step - loss: 0.6089 - accuracy: 0.6834 - val_loss: 0.7779 - val_accuracy: 0.6654


In [9]:
#save model
model.save_weights("model.h5")

In [8]:
# prepare test
test_filenames = os.listdir("./dataset/test_set")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]
test_df

,filename
0,cat.4001.jpg
1,cat.4002.jpg
2,cat.4003.jpg
3,cat.4004.jpg
4,cat.4005.jpg
...,...
1995,dog.4996.jpg
1996,dog.4997.jpg
1997,dog.4998.jpg
1998,dog.4999.jpg


In [9]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "./dataset/test_set", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=Image_Size,
    batch_size=batch_size,
    shuffle=False
)
test_df

Found 2000 validated image filenames.


,filename
0,cat.4001.jpg
1,cat.4002.jpg
2,cat.4003.jpg
3,cat.4004.jpg
4,cat.4005.jpg
...,...
1995,dog.4996.jpg
1996,dog.4997.jpg
1997,dog.4998.jpg
1998,dog.4999.jpg


In [11]:
predict = model.predict(test_generator, steps=np.ceil(nb_samples/batch_size))

In [12]:
test_df['category'] = np.argmax(predict,axis=-1)
test_df.head()




,filename,category
0,cat.4001.jpg,1
1,cat.4002.jpg,1
2,cat.4003.jpg,1
3,cat.4004.jpg,1
4,cat.4005.jpg,1
...,...,...
1995,dog.4996.jpg,1
1996,dog.4997.jpg,1
1997,dog.4998.jpg,1
1998,dog.4999.jpg,1
